In [21]:
import sys
sys.path.insert(0, '..')
from net_framework import *
import pandas as pd

In [22]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [23]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

<ipython-input-23-f811cbef4bec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
<ipython-input-23-f811cbef4bec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
<ipython-input-23-f811cbef4bec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [24]:
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]
print(chip_num)

[141, 274, 129, 230, 302, 324, 27, 290, 92, 66, 83, 22, 242, 106, 177, 260, 206, 52, 199, 95, 293, 219, 108, 330, 221, 284, 327, 244, 294, 272, 172, 158, 120, 286, 262, 107, 241, 319, 224, 314, 252, 44, 16, 217, 201, 84, 56, 237, 6, 159, 174, 297, 96, 12, 128, 214, 124, 306, 149, 251, 67, 229, 5, 196, 100, 2, 31, 175, 74, 116, 264, 189, 303, 142, 70, 139, 59, 15, 78, 285, 190, 160, 292, 300, 63, 151, 35, 278, 318, 212, 187, 110, 326, 146, 258, 193, 91, 143, 33, 315, 268, 29, 161, 145, 253, 82, 289, 186, 49, 200, 104, 138, 322, 254, 102, 213, 167, 99, 152, 215, 86, 118, 239, 126, 263, 256, 176, 17, 121, 125, 261, 270, 48, 181, 42, 275, 218, 236, 81, 7, 282, 45, 180, 310, 119, 147, 246, 136, 55, 40, 211, 19, 153, 1, 37, 233, 62, 308, 197, 273, 32, 68, 28, 21, 173, 46, 321, 298, 288, 3, 76, 30, 185, 208, 287, 58, 225, 320, 304, 280, 195, 20, 202, 85, 299, 223, 301, 316, 238, 77, 307, 188, 169, 34, 164, 291, 220, 247, 192, 23, 231, 269, 203, 207, 65, 112, 154, 98, 148, 245, 132, 266, 9, 32

In [25]:
l1 = term_data[term_data.get('#Lnum').eq(1)]
unique_symbols = list(l1['Term Abbrev'].unique())
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [26]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 1
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)
chip_train = []
#pull the percentage for each cnum
for x in chip_num:
    chip_train.append((l1_result.loc[l1["#cnum"]==x]).values.tolist()[0])
#check
print(chip_train)

         LB    LE    WK    LF     F     G     S   GB    FU
#cnum                                                     
1      0.36  0.00  0.00  0.04  0.08  0.52  0.00  0.0  0.00
2      0.12  0.00  0.04  0.12  0.60  0.04  0.00  0.0  0.08
3      0.00  0.96  0.04  0.00  0.00  0.00  0.00  0.0  0.00
4      0.28  0.00  0.40  0.00  0.08  0.04  0.20  0.0  0.00
5      0.04  0.00  0.00  0.20  0.68  0.08  0.00  0.0  0.00
...     ...   ...   ...   ...   ...   ...   ...  ...   ...
326    0.08  0.20  0.36  0.00  0.20  0.00  0.12  0.0  0.04
327    0.04  0.00  0.00  0.68  0.20  0.04  0.04  0.0  0.00
328    0.64  0.00  0.08  0.00  0.00  0.20  0.08  0.0  0.00
329    0.08  0.08  0.40  0.00  0.12  0.00  0.28  0.0  0.04
330    0.04  0.00  0.00  0.72  0.24  0.00  0.00  0.0  0.00

[330 rows x 9 columns]
[[0.44, 0.0, 0.0, 0.04, 0.0, 0.52, 0.0, 0.0, 0.0], [0.04, 0.16, 0.24, 0.0, 0.28, 0.08, 0.2, 0.0, 0.0], [0.0, 0.0, 0.2, 0.08, 0.48, 0.04, 0.12, 0.0, 0.08], [0.08, 0.0, 0.04, 0.16, 0.64, 0.0, 0.08, 0.0, 0.0], [0

In [27]:
#Number of training iterations
num_iters = 200
#Size of training dataset
num_examples = 2000
#Listing out the shapes of each model
colors_num = len(chip_train[0])
input_size = 3

network_shapes = [(input_size, [100], colors_num), (input_size, [100], colors_num), (input_size, [100], colors_num)]
#Learning rate of the network
rate = 0.001

#Generating Training Data
input_num = torch.FloatTensor(lab_norm)
output_num = torch.FloatTensor(chip_train)
#Array of losses over training period for each network
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    

    for i in range(num_iters):
        #Calculating l1 error
        error = NN.l1error(output_num, NN(input_num))
        if i == 0: 
            dh = display("#" + str(i) + " Error: " + str(error), display_id=True)
        else:
            dh.update("#" + str(i) + " Error: " + str(error))
            
        NN.train(input_num, output_num)
    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num));


Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#199 Error: 0.025111332535743713'

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#199 Error: 0.023220110684633255'

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9


'#199 Error: 0.024023910984396935'

In [28]:
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    
    #Loading the network we trained in the prev. section
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    NN.load_state_dict(torch.load("saved_networks/Net " + str(net_num)))
    validation_error = NN.l1error(output_num, torch.round(NN(input_num)))

    print("The validation error is: " + str(validation_error), flush = True)  

Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9
The validation error is: 0.08771716803312302
Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9
The validation error is: 0.08773063868284225
Network Shape:
Input Size = 3
Hidden Size = [100]
Output Size = 9
The validation error is: 0.08801345527172089
